# Importing the necessary libraries
- `!pip install pandas spotipy json tqdm sqlite3`
- note spotipy.oauth2 as oauth2, and SpotifyClientCredentials were imported

In [105]:
import pandas as pd 
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyClientCredentials
import json 
import os
from tqdm import tqdm
import sqlite3
import time

In [155]:
credentials_file_path = "../credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [156]:
ccm=SpotifyClientCredentials(client_id= credentials['client_id'],
    client_secret= credentials['client_secret'])

spotify = spotipy.Spotify(client_credentials_manager=ccm)

# Defining Functions

In [154]:

#obtaining the artist ids through the data we obtain from load_files
#returns a unique list of artist ids to be used by get_info
def get_artist_ids(data): 
        
        
        artist_ids = [track['track']['artists'][0]['uri'] for track in data['tracks']['items']]

        return list(set(artist_ids))


#function gets info about the speific artist through their id and returns the genres. 
def get_info(artist_ids):
     #we will have to loop through the list before calling this function so it applies to each playlist. 
     return [spotify.artist(artist)['genres'] for artist in artist_ids]


#loading the json files through a given folder_path
def load_files(folder_path):
    json_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.json')]


    raw_data_list = []
    for file_path in json_files:
        with open(file_path, 'r') as file:
            raw_data_list.append(json.load(file))
    return raw_data_list

# Uploading to SQl
## _Obtaining Genres of artists_

In [170]:
#loading the datafiles through load_files
data_files = load_files('../data/raw_data/us_playlist')

In [171]:
#obtaining the artist ids in each playlist and appending it to another list 
#artist_ids = []
#for i in tqdm(data_files): 
   # artist_ids.append(get_artist_ids(i))


#using list comprehension instead of for loops 
artist_ids = [get_artist_ids(i) for i in tqdm(data_files)]

100%|██████████| 20/20 [00:00<00:00, 10765.67it/s]


In [172]:
#obtains the info from the specific artist in each playist and appends to a list

#list_of_genres = []
#for i in tqdm(artist_ids):
   # list_of_genres.append(get_info(i))

#using list comprehension instead of for loops 
list_of_genres = [get_info(i) for i in tqdm(artist_ids)]

100%|██████████| 20/20 [03:00<00:00,  9.03s/it]


In [173]:
#Cleaning up the list 
#categories = []
#for i in list_of_genres:
    #categories.append(list(set([genre for genre_list in i for genre in genre_list])))

#using list comprehension instead of for loops 
categories = [list(set([genre for genre_list in i for genre in genre_list])) for i in list_of_genres]

In [174]:
#creates a dictionary with the name of the eplaylist and its id
fixed_dict = {data_names['name']: data_names['uri'] for data_names in data_files}

#gets the uri of each playlist 
list_of_uri = list(fixed_dict.values())

#gets the name of each playlist
playlist = list(fixed_dict.keys())

#getting the followers of all the playlists
number_of_followers = [followers['followers']['total'] for followers in data_files]

## Creating DataFrame
### playlist dataframe 

In [175]:
#create dataframe with playlist name uri and followers
df_playlist = pd.DataFrame({'playlist_name': playlist, 'uri': list_of_uri, 'followers': number_of_followers}).reset_index()

#renames the columns 
df_playlist = df_playlist.rename(columns={'index': 'id'})

#offsets the index by 1 
df_playlist['id'] = df_playlist['id'].apply(lambda x: x+1)

#adding column country id to the dataframe as c*
df_playlist['country_id'] = ['c3'] * len(df_playlist)

### category dataframe

In [176]:
#create dataframe with playlist category and id 
playlist_category = pd.DataFrame({'categories': categories}).reset_index()

#renames the columns
playlist_category = playlist_category.rename(columns = {'index': 'playlist_id'})

#offsets the id column by 1 
playlist_category['playlist_id'] = playlist_category['playlist_id'].apply(lambda x: x+1)

#explodes the categories so we can upload to sql database
playlist_category = playlist_category.explode('categories')

#adding column country id to the dataframe as c3. 
playlist_category['country_id'] = ['c3'] * len(playlist_category)

playlist_category

,playlist_id,categories,country_id
0,1,color noise,c3
0,1,water,c3
0,1,rain,c3
0,1,binaural,c3
0,1,environmental,c3
...,...,...,...
19,20,urban contemporary,c3
19,20,new jersey underground rap,c3
19,20,queens hip hop,c3
19,20,k-pop girl group,c3


In [165]:
country = {
    'country': ['United Kingdom', 'United States', 'Mexico'],
    'id': ['c1', 'c2', 'c3']
}
country_df = pd.DataFrame(country)

In [125]:
%reload_ext sql
%config SqlMagic.autocommit=True

## _Establishing Connection to SQL_

In [127]:
%sql sqlite:///../data/clean/playlist.db --alias playlist

Connecting to 'playlist'

In [128]:
DATA_FOLDER = os.path.join('../data/clean/')

In [129]:
conn = sqlite3.connect(os.path.join(DATA_FOLDER, 'playlist.db'))

### Creating Tables

In [130]:
%%sql

    -- creating the tables in order for analysis 

DROP TABLE IF EXISTS playlist_name; 

CREATE TABLE playlist_name (

 "id" INTEGER ,
 "playlist_name" VARCHAR(50),
 "uri" VARCHAR(50),
 "Followers" INTEGER,
 "country_id" VARCHAR(50),
 FOREIGN KEY("country_id") REFERENCES "country" ("id")
 PRIMARY KEY("id", "country_id")
);

DROP TABLE IF EXISTS playlist_category; 

CREATE TABLE playlist_category(

    "playlist_id" INTEGER,
    "categories" VARCHAR(50),
    "country_id" VARCHAR(50),
    FOREIGN KEY("playlist_id") REFERENCES "playlist_name" ('id'),
    FOREIGN KEY("country_id") REFERENCES "country" ("id")
); 

DROP TABLE IF EXISTS country; 

CREATE TABLE country (
    "id" VARCHAR(50) PRIMARY KEY,
    "country" VARCHAR(50)    
);

Running query in 'playlist'

++
||
++
++

### Uploading data

In [166]:
#uploading data
df_playlist.to_sql('playlist_name', conn, if_exists='append', index=False)

playlist_category.to_sql('playlist_category', conn, if_exists='append', index=False)

country_df.to_sql('country', conn, if_exists = 'replace', index = False)



#I mighthave to create a dataframe after we ahve all the data. 



3

In [169]:
#closing the connection
conn.close()